# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, AtomsBase.Periodic, box=[[0.0, 2.715, 2.715], [2.715, 0.0, 2.715], [2.715, 2.715, 0.0]]u"Å")

To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.921683936351                   -0.69    5.9
  2   -7.926168063223       -2.35       -1.22    1.0
  3   -7.926838020194       -3.17       -2.37    1.9
  4   -7.926861466831       -4.63       -3.04    2.5
  5   -7.926861646716       -6.75       -3.42    1.9
  6   -7.926861670601       -7.62       -3.82    1.6
  7   -7.926861679924       -8.03       -4.24    1.4
  8   -7.926861681679       -8.76       -4.73    1.9
  9   -7.926861681858       -9.75       -5.25    1.9
 10   -7.926861681870      -10.90       -5.72    2.0
 11   -7.926861681873      -11.67       -6.35    1.5
 12   -7.926861681873      -12.81       -7.58    2.1
 13   -7.926861681873      -14.35       -8.04    4.1


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),

which directly yields an AbstractSystem.

In [5]:
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.921676651836                   -0.69    6.0
  2   -7.926164500558       -2.35       -1.22    1.0
  3   -7.926837776982       -3.17       -2.37    1.8
  4   -7.926861517394       -4.62       -3.04    2.9
  5   -7.926861646059       -6.89       -3.42    1.6
  6   -7.926861670683       -7.61       -3.82    1.9
  7   -7.926861680402       -8.01       -4.32    1.1
  8   -7.926861681739       -8.87       -4.81    2.1
  9   -7.926861681858       -9.92       -5.23    1.6
 10   -7.926861681871      -10.88       -5.83    1.9
 11   -7.926861681873      -11.77       -7.12    1.9
 12   -7.926861681873      -13.29       -7.65    3.5
 13   -7.926861681873      -14.35       -7.92    2.6
 14   -7.926861681873   +  -14.45       -8.88    1.2


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [6]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.921681000204                   -0.69    5.9
  2   -7.926165434503       -2.35       -1.22    1.0
  3   -7.926840770348       -3.17       -2.37    1.6
  4   -7.926864927929       -4.62       -3.04    3.1
  5   -7.926865054054       -6.90       -3.40    1.5
  6   -7.926865080389       -7.58       -3.77    2.0
  7   -7.926865091651       -7.95       -4.35    1.1


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [7]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, AtomsBase.Periodic, box=[[0.0, 5.13, 5.13], [5.13, 0.0, 5.13], [5.13, 5.13, 0.0]]u"a₀")

Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [8]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, AtomsBase.Periodic, box=[[0.0, 5.131551291421279, 5.131551291421279], [5.131551291421279, 0.0, 5.131551291421279], [5.131551291421279, 5.131551291421279, 0.0]]u"a₀")